### Data Loading

In [1]:
import torch
import torch.serialization
import torch_geometric

torch.serialization.add_safe_globals([
    torch_geometric.data.Data,
    torch_geometric.data.data.DataEdgeAttr,
    torch_geometric.data.data.DataTensorAttr,
    torch_geometric.data.storage.GlobalStorage
])


# Import from the same folder as notebook
from data_loading.parts_dataset import PartGraphDataset

PART_GRAPHS = "/media/swapnil/3f73cc1a-8f9d-4c19-87af-99b3512ff5b2/MK_S/graphs/parts"

ds = PartGraphDataset(PART_GRAPHS)
print("Dataset size:", len(ds))

# Triplet test
a, p, n = ds[0]
print(a)
print(p)
print(n)


/home/swapnil/anaconda3/envs/occ310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded 447416 part graphs
103409 unique part IDs
Dataset size: 447416
Data(x=[50, 14], edge_index=[2, 240], step_path='/media/swapnil/3f73cc1a-8f9d-4c19-87af-99b3512ff5b2/MK_S/step/cd5db363d1aeb0e697b3e1e4_13a78123a4ddb1a9255697e4_5c474d2e271d4e29a39ead53_default_kizeisa.step')
Data(x=[35, 14], edge_index=[2, 148], step_path='/media/swapnil/3f73cc1a-8f9d-4c19-87af-99b3512ff5b2/MK_S/step/cd5db363d1aeb0e697b3e1e4_13a78123a4ddb1a9255697e4_5c474d2e271d4e29a39ead53_default_kjqumsa.step')
Data(x=[554, 14], edge_index=[2, 2976], step_path='/media/swapnil/3f73cc1a-8f9d-4c19-87af-99b3512ff5b2/MK_S/step/df8a661827de45b4121828f1_97ee2483e40d5ef9eac74309_c99ee4d81961c4979eaeea0b_default_jjeei.step')


### Testing model before training

In [ ]:
from models.part_gnn import PartGNN
import torch

# One sample graph from your dataset
a, _, _ = ds[0]

model = PartGNN(in_channels=a.x.shape[1], embed_dim=256)
print(model)

# Fake batch dimension
batch = torch.zeros(a.x.size(0), dtype=torch.long)

out = model(a.x, a.edge_index, batch)
print("Embedding shape:", out.shape)


PartGNN(
  (conv1): GINConv(nn=Sequential(
    (0): Linear(in_features=14, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
  ))
  (conv2): GINConv(nn=Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=256, bias=True)
  ))
  (ln): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Embedding shape: torch.Size([1, 256])


### running the training

In [4]:
import os
os.listdir('training')

['training_partgnn.py']

### After embeddings are created /project/part_embeddings_struct.py

/home/swapnil/anaconda3/envs/occ310/lib/python3.10/site-packages/torch/__config__.py:9: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:119.)
  return torch._C._show_config()
/home/swapnil/anaconda3/envs/occ310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data(x=[15, 8], edge_index=[2, 28], edge_attr=[28, 1], part_ids=[15], assembly_path='/media/swapnil/3f73cc1a-8f9d-4c19-87af-99b3512ff5b2/MK_S/Automate/assemblies/asm_00000/01984c6ecc641bbea1b098c1_7cf8f9bc95fd3a123d51b6d4_d8e061ba74a1304ea959b3f8_default.json', assembly_id='01984c6ecc641bbea1b098c1_7cf8f9bc95fd3a123d51b6d4_d8e061ba74a1304ea959b3f8_default')


### plug embeddings into the AssemblyGraphDataset

In [2]:
import torch
from assembly_graph_pyg import AssemblyGraphDataset

part_embeddings = torch.load("part_embeddings.pt", map_location="cpu")

ds = AssemblyGraphDataset(
    "assembly_filter_out/assembly_graphs_v1.jsonl",
    part_embeddings=part_embeddings
)

d0 = ds[0]
print(d0)
print("x dim:", d0.x.shape[1])


Data(x=[15, 8], edge_index=[2, 28], edge_attr=[28, 1], part_ids=[15], assembly_path='/media/swapnil/3f73cc1a-8f9d-4c19-87af-99b3512ff5b2/MK_S/Automate/assemblies/asm_00000/01984c6ecc641bbea1b098c1_7cf8f9bc95fd3a123d51b6d4_d8e061ba74a1304ea959b3f8_default.json', assembly_id='01984c6ecc641bbea1b098c1_7cf8f9bc95fd3a123d51b6d4_d8e061ba74a1304ea959b3f8_default')
x dim: 8


### do all assemblies have embeddings coverage?

If any part id is missing in embeddings, dataset loading will throw an error.
Run this quick coverage test on the first 200 assemblies:

In [3]:
import json
from pathlib import Path

GRAPHS = Path("assembly_filter_out/assembly_graphs_v1.jsonl")

missing_parts = 0
checked = 0

with GRAPHS.open() as f:
    for i, line in enumerate(f):
        if i >= 200:
            break
        rec = json.loads(line)
        assembly_path = rec["assembly_path"]

        assembly = json.load(open(assembly_path))
        for p in assembly.get("parts", []):
            pid = p.get("id")
            if isinstance(pid, str) and pid not in part_embeddings:
                missing_parts += 1

        checked += 1

print("Checked assemblies:", checked)
print("Missing part embeddings (count):", missing_parts)


Checked assemblies: 200
Missing part embeddings (count): 0
